# Visualizing Data from BACnet Simulator via Python

### 1. Starting

- Download Windows YABE from: https://sourceforge.net/projects/yetanotherbacnetexplorer/
- Once installed, go to Windows Start button > All Programs > Yabe folder and run Yabe (with a magnifying glass logo) and Bacnet.Room.Simulator app.
- Connect the simulator by clicking on the green plus logo (Add Channel) and clickingon the Start button corresponding to BAC0 port.
- Double click on the Device onject to display the object's items.
- Click on drag on any of the objects into the Subscriptions, Periodic Polling, Events/Alarms tab to display its values.
- Check on all or any of the devices to show the plots. 

<img src="Yabe_explorer.png">
<br>
<img src="room_control_simulator.png">
<br>

- Start a jupyter notebook on your computer via anaconda prompt.

reference: 

- code is adapted from: https://github.com/techbeast-org/bacnet-ip/tree/master
- instruction video: https://www.youtube.com/watch?v=TyEXDnjBsD8

### 2. Install the necessary libraries:

In [1]:
!pip install BAC0==21.12.3 bacpypes==0.18.6 colorama==0.4.4

In [1]:
#update a compatible pandas library
!pip uninstall -y pandas
!pip install pandas==1.3.5

  Using cached pandas-1.3.5-cp37-cp37m-win32.whl (8.9 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydexp 1.0.0 requires numpy<=1.21.2,>=1.18.0, but you have numpy 1.21.6 which is incompatible.
pydexp 1.0.0 requires pandas<=1.3.3,>=1.0.0, but you have pandas 1.3.5 which is incompatible.


In [16]:
#Install plotly
!pip install plotly

### 3. Communicate with Yabe and display a live data in a graph.

Make sure to input your own local ip address and specify the device's BACnet/I UDP port number (obtained from Yabe explorer):
- 192.168.1.66 → the device’s IP address
- 58956 → the device’s BACnet/IP UDP port

In [12]:
import BAC0
import time
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Setup BACnet connection
try:
    bacnet = BAC0.lite(ip="192.168.1.66/24", port=47818)
    print("✅ BACnet connected.")
except Exception as e:
    print(f"[ERROR] Failed to start BACnet: {e}")
    exit()

# Buffer for plotting
history = {
    "Analog_input0": [],
    "Analog_input1": [],
    "Analog_input2": [],
    "time": []
}

# --- Read Analog Inputs from BACnet ---
def read_bacnet_inputs():
    values = {}
    for i in range(3):
        point = f"192.168.1.66:55099 analogInput {i} presentValue"
        try:
            value = float(bacnet.read(point))
        except Exception as e:
            print(f"[ERROR] reading analogInput {i}: {e}")
            value = None
        values[f"Analog_input{i}"] = value
    return values

# --- Plot the Live Graph ---
def plot_live():
    try:
        fig = go.Figure()
        for key in ["Analog_input0", "Analog_input1", "Analog_input2"]:
            fig.add_trace(go.Scatter(
                x=history["time"], y=history[key],
                mode='lines+markers', name=key
            ))
        fig.update_layout(
            title="Live BACnet Analog Inputs",
            xaxis_title="Time (s)",
            yaxis_title="Value",
            template="plotly_dark",
            height=500
        )
        clear_output(wait=True)
        display(fig)
    except Exception as e:
        print(f"[ERROR] plotting: {e}")

# --- Main Loop to Read and Plot ---
def main():
    print("📈 Starting live monitoring...")
    start = time.time()
    try:
        while True:
            values = read_bacnet_inputs()
            now = round(time.time() - start, 1)
            history["time"].append(now)
            for key in ["Analog_input0", "Analog_input1", "Analog_input2"]:
                history[key].append(values.get(key))
            plot_live()
            time.sleep(2)
    except KeyboardInterrupt:
        print("\n🛑 Monitoring stopped by user.")
    except Exception as e:
        print(f"[ERROR] main loop: {e}")
    finally:
        try:
            bacnet.disconnect()
            print("🔌 BACnet connection closed.")
        except Exception as e:
            print(f"[ERROR] during disconnect: {e}")

# Run only if called directly
if __name__ == '__main__':
    main()



🛑 Monitoring stopped by user.
🔌 BACnet connection closed.


output example: 

<img src="plot_example.png">

### Review bacnet object

In [ ]:
import BAC0
import time
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Setup BACnet
bacnet = BAC0.lite(ip="192.168.1.66/24", port=47819)

In [34]:
#dir(bacnet)

In [7]:
help(bacnet)

Help on Lite in module BAC0.scripts.Lite object:

class Lite(BAC0.scripts.Base.Base, BAC0.core.functions.Discover.Discover, BAC0.core.io.Read.ReadProperty, BAC0.core.io.Write.WriteProperty, BAC0.core.io.Simulate.Simulation, BAC0.core.functions.TimeSync.TimeSync, BAC0.core.functions.Reinitialize.Reinitialize, BAC0.core.functions.DeviceCommunicationControl.DeviceCommunicationControl, BAC0.core.functions.cov.CoV, BAC0.core.functions.Schedule.Schedule, BAC0.core.functions.Calendar.Calendar, BAC0.core.functions.Text.TextMixin)
 |  Lite(ip=None, port=None, mask=None, bbmdAddress=None, bbmdTTL=0, bdtable=None, ping=True, ping_delay=300, db_params=None, **params)
 |  
 |  Build a BACnet application to accept read and write requests.
 |  [Basic Whois/IAm functions are implemented in parent BasicScript class.]
 |  Once created, execute a whois() to build a list of available controllers.
 |  Initialization requires information on the local device.
 |  
 |  :param ip='127.0.0.1': Address must be i

### Explore bacnet object

In [11]:
import BAC0
import time
from datetime import datetime

# ======================== Setup BACnet ========================
try:
    bacnet = BAC0.lite(ip="192.168.1.66/24", port=47818)
    print("✅ BACnet connected.\n")
except Exception as e:
    print(f"[ERROR] Failed to connect: {e}")
    exit()

# ===================== BACnet Utility Functions =====================

def discover_devices(bacnet):
    print("🔎 Discovering devices with WHOIS...")
    try:
        bacnet.whois()
        devices = bacnet.devices
        if isinstance(devices, list) and len(devices) == 0:
            print("⚠️  No devices discovered via broadcast.")
        else:
            print("\n📋 Discovered Devices:")
            print(devices)
    except Exception as e:
        print(f"[ERROR] discover_devices: {e}")

def summarize_devices(bacnet):
    print("\n📦 Device Summary:")
    try:
        devices = bacnet.devices
        if isinstance(devices, list):
            for device in devices:
                name = device.get('Name', 'Unknown')
                instance = device.get('Instance', 'N/A')
                address = device.get('Address', 'N/A')
                vendor = device.get('Vendor', 'N/A')
                print(f"🟢 {name} | Instance: {instance} | Address: {address} | Vendor: {vendor}")
        else:
            print("⚠️  Devices list is not in expected format.")
    except Exception as e:
        print(f"[ERROR] summarize_devices: {e}")

def list_known_networks(bacnet):
    print("\n🌐 Known Network Numbers:")
    try:
        known = bacnet.known_network_numbers
        print(known if known else "⚠️  No known networks.")
    except Exception as e:
        print(f"[ERROR] list_known_networks: {e}")

def show_routing_table(bacnet):
    print("\n🧭 BACnet Routing Table:")
    try:
        routes = bacnet.routing_table
        if routes:
            for router, info in routes.items():
                print(f"🔁 Router {router} handles networks: {info['networks']}")
        else:
            print("⚠️  No routing info available.")
    except Exception as e:
        print(f"[ERROR] show_routing_table: {e}")

def discover_object_types(bacnet, address):
    print(f"\n🔍 Object List from {address}")
    try:
        object_list = bacnet.read(f"{address} device 1 objectList")
        print(f"📄 Object List: {object_list}")
    except Exception as e:
        print(f"[ERROR] discover_object_types: {e}")

def read_analog_inputs(bacnet, address, max_inputs=3):
    print(f"\n📖 Reading Analog Inputs from {address}")
    for i in range(max_inputs):
        try:
            val = bacnet.read(f"{address} analogInput {i} presentValue")
            print(f"AI:{i} = {val}")
        except Exception as e:
            print(f"[ERROR] reading AI:{i}: {e}")

def test_read_status(bacnet, address):
    print(f"\n📶 Testing device at {address} via presentValue read...")
    try:
        val = bacnet.read(f"{address} analogInput 0 presentValue")
        print(f"✅ Device responded with AI:0 value = {val}")
    except Exception as e:
        print(f"[ERROR] Device read test failed: {e}")

def test_time_sync(bacnet, address):
    print(f"\n⏱️  Testing time sync on {address}...")

    try:
        try:
            time_before = bacnet.read(f"{address} device 1 localTime")
            date_before = bacnet.read(f"{address} device 1 localDate")
            print(f"📆 Device time before sync: {date_before} {time_before}")
        except Exception as e:
            print(f"⚠️  Cannot read device clock: {e}")

        try:
            bacnet.time_sync(destination=address)
            print("✅ time_sync() command sent.")
        except Exception as e:
            print(f"⚠️  time_sync() failed: {e}")

        try:
            time_after = bacnet.read(f"{address} device 1 localTime")
            date_after = bacnet.read(f"{address} device 1 localDate")
            print(f"📆 Device time after sync: {date_after} {time_after}")
        except:
            print("⏱️  Post-sync time read skipped (not supported).")

    except Exception as e:
        print(f"[ERROR] test_time_sync: {e}")

def explore_all_objects(bacnet, address):
    print(f"\n🧠 Exploring all objects at {address}")
    try:
        object_list = bacnet.read(f"{address} device 1 objectList")
    except Exception as e:
        print(f"[ERROR] Failed to get object list: {e}")
        return

    for obj_type, instance in object_list:
        print(f"\n🔎 {obj_type} {instance}")
        for prop in ['presentValue', 'description', 'units', 'statusFlags', 'objectName']:
            try:
                value = bacnet.read(f"{address} {obj_type} {instance} {prop}")
                print(f"  - {prop}: {value}")
            except:
                pass  # Skip unsupported properties silently

# ========================== Run All ==========================

def main():
    discover_devices(bacnet)
    summarize_devices(bacnet)
    list_known_networks(bacnet)
    show_routing_table(bacnet)

    address = "192.168.1.66:55099"

    discover_object_types(bacnet, address)
    read_analog_inputs(bacnet, address)
    test_read_status(bacnet, address)
    test_time_sync(bacnet, address)
    explore_all_objects(bacnet, address)

    bacnet.disconnect()
    print("\n🔌 BACnet disconnected. All tasks complete.")

if __name__ == "__main__":
    main()


✅ BACnet connected.

🔎 Discovering devices with WHOIS...
⚠️  No devices discovered via broadcast.

📦 Device Summary:

🌐 Known Network Numbers:
⚠️  No known networks.

🧭 BACnet Routing Table:
⚠️  No routing info available.

🔍 Object List from 192.168.1.66:55099
📄 Object List: [('device', 1392679), ('analogInput', 0), ('analogInput', 1), ('analogInput', 2), ('analogValue', 0), ('analogValue', 1), ('analogValue', 2), ('analogValue', 3), ('characterstringValue', 1), ('binaryValue', 0), ('binaryValue', 1), ('multiStateValue', 0), ('multiStateValue', 1)]

📖 Reading Analog Inputs from 192.168.1.66:55099
AI:0 = 69.5999984741211
AI:1 = 101.30000305175781
AI:2 = 53.599998474121094

📶 Testing device at 192.168.1.66:55099 via presentValue read...
✅ Device responded with AI:0 value = 69.5999984741211

⏱️  Testing time sync on 192.168.1.66:55099...
⚠️  Cannot read device clock: APDU Abort Reason : other
✅ time_sync() command sent.
⏱️  Post-sync time read skipped (not supported).

🧠 Exploring all obj

### Useful tutorials on BACnet:

- Part 1-What is BACnet: https://www.youtube.com/watch?v=oevGXrkxEos
- Part 2-Device modeling: https://www.youtube.com/watch?v=DEzpUCC3TDI
- Part 3-Network Types: https://www.youtube.com/watch?v=cM4KVD1119o
- Part 4-BACnet services: https://www.youtube.com/watch?v=a3TuEypguy8
- Part 5-Interoperability: https://www.youtube.com/watch?v=-th7dJsrn74
- Part 6-BACnet BIBBs: https://www.youtube.com/watch?v=hgO9ae-bTIg
- Part 7-BACnet Device Profiles: https://www.youtube.com/watch?v=A1gAntcENFI
- Part 8-BACnet compliance: https://www.youtube.com/watch?v=zVa-Xb8tTDc